# Steam Data Cleaning (Part 3)

*This is part of a larger series of notebooks on downloading, processing and analysing data from the steam store. [See all notebooks here.](../notebooks)*

# Release data optimisation and combining

**TODO**: genre and categories section writeup

Currently our downloaded data is not in a very usable or useful state. Many of the columns contain lengthy strings or missing values, both of which are crippling to analysis and especially to any machine learning techniques we may wish to implement.

The main aims of this project are to investigate various sales and play-time statistics for games from the steam store, and see how different features of games may have an effect on the success of those games. Keeping this in mind will help inform our decisions about how to handle the various columns in our data set, however it may be a good idea to keep columns which may not seem useful to this particular project in order to provide a robust data set for future analysis projects.

To begin with, we'll import our libraries and set some options, then take a look at the downloaded data from the steam api. Once that is taken care of we will move on to the steamspy data and repeat the process. Hopefully by the end we will have clean data sets to use in the next step, exploratory analysis and visualisation.

### Aims:
- Improve functions
- Prepare notebook for delivery

### (Raw) Data Dictionary

Sort out data dictionary  

API and data dictionary:
https://steamspy.com/api.php

### Future ideas:
- pc requirements analysis over time
- picture analysis
- keyword/recommender analysis
- categories could make table in a database all on its own, perhaps in future
- for genres (and categories?) could create main genre, selected from list of key genres, allowing hybrids like action_adventure if contains both
- remove titles over £60/100?

In [2]:
# load extensions and magics

# http://raw.github.com/jrjohansson/version_information/master/version_information.py
%load_ext version_information
%reload_ext version_information

%version_information numpy, pandas

Software versions
Python 3.7.3 64bit [MSC v.1900 64 bit (AMD64)]
IPython 7.5.0
OS Windows 10 10.0.17763 SP0
numpy 1.16.3
pandas 0.24.2
Fri May 24 12:07:27 2019 GMT Summer Time

In [3]:
# import libraries
from ast import literal_eval
import itertools
import time
import re

import numpy as np
import pandas as pd

In [4]:
# customisations
pd.set_option("max_columns", 100)
# pd.reset_option("max_columns")

## Cleaning steam data

### Import Data

We begin by importing the raw steam data we generated previously in data collection, which can be viewed by following the link to `../deliver/1-data-collection.ipynb` below. From a quick inspection of the data, we can see that we have a mixture of numeric and string columns, plenty of missing values, and a number of columns stored as dictionaries.

In [7]:
from IPython.display import FileLink
FileLink("../notebooks/1-data-collection.ipynb")

C:\Users\nikda\OneDrive\Documents\ds\projects\2019-04-23-steam\notebooks\1-data-collection.ipynb

In [8]:
raw_steam_data = pd.read_csv('../data/raw/steam_app_data.csv')

print('Rows:', raw_steam_data.shape[0])
print('Columns:', raw_steam_data.shape[1])
raw_steam_data.head()

Rows: 29235
Columns: 39


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 719, 'final': 7...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 65735},{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2802},{'total': 0},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 1992},{'total': 0},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</s

We can chain the `isnull()` and `sum()` methods to easily see how many missing values we have in each column. Immediately we can see that a number of columns have over 20,000 rows with missing data, and in a data set of almost 30,000 rows these are unlikely to provide any useful information.

In [9]:
raw_steam_data.isnull().sum()

type                         149
name                           1
steam_appid                    0
required_age                 149
is_free                      149
controller_support         23237
dlc                        24260
detailed_description         175
about_the_game               175
short_description            175
fullgame                   29235
supported_languages          163
header_image                 149
website                     9983
pc_requirements              149
mac_requirements             149
linux_requirements           149
legal_notice               19168
drm_notice                 29077
ext_user_account_notice    28723
developers                   264
publishers                   149
demos                      27096
price_overview              3712
packages                    3370
package_groups               149
platforms                    149
metacritic                 26254
reviews                    23330
categories                   714
genres    

### Processing and Optimising Release Date

The final column to clean, release date, provides some interesting optimisation and learning challenges. We've encountered some columns with a similar structure already, so we can use what we've learned so far, but now we have some dates to handle.

First we shall inspect the raw format of the column. As we can see below, it is stored as a dictionary-like string object containing values for `coming_soon` and `date`. From the first few rows it would appear that the dates are stored in a uniform format - day as an integer, month as a 3-character string abbreviation, a comma, then the year as a four-digit number. We can parse this either using the python built-in datetime module, or as we already have pandas imported, we can use the [pd.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) function.

Also, as our analysis will involve looking at ownership and sales data, looking at games that are not released yet will not be useful to us. Intuitively, we can drop any titles which are marked as coming soon, presumably having this value set to true. As a side note, once parsed it may be worth checking that no release dates in our data are beyond the current date, just to make doubly sure none slip through.

In [92]:
display(raw_steam_data['price_overview'][0])
display(raw_steam_data['release_date'][0])

"{'currency': 'GBP', 'initial': 719, 'final': 719, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '£7.19'}"

"{'coming_soon': False, 'date': '1 Nov, 2000'}"

In [93]:
steam_data[['name', 'release_date']].head()

,name,release_date
0,Counter-Strike,"{'coming_soon': False, 'date': '1 Nov, 2000'}"
1,Team Fortress Classic,"{'coming_soon': False, 'date': '1 Apr, 1999'}"
2,Day of Defeat,"{'coming_soon': False, 'date': '1 May, 2003'}"
3,Deathmatch Classic,"{'coming_soon': False, 'date': '1 Jun, 2001'}"
4,Half-Life: Opposing Force,"{'coming_soon': False, 'date': '1 Nov, 1999'}"


As usual, one of the first steps we'll take is to check for null values. Luckily, it seems that the cleaning we have performed already has removed any null values from our data set, as seen below. We may still have some hidden empty values of course. 

In [94]:
print('Null values:\n')
print('Raw data:', raw_steam_data['release_date'].isnull().sum())
print('Partially cleaned:', steam_data['release_date'].isnull().sum())

Null values:

Raw data: 149
Partially cleaned: 0


Exploring the data using the value_counts method brings a couple of data issues to light.

In the raw data, we can see that 64 rows have data but date is an empty string, ''. Like we've seen before, this means they do not have null values, but may need to be treated as such depending on the reason. This may be data corruption, or it may be another reason entirely. We will probably have to decide what to do with these cases and investigate further.

Another issue we can notice is that while most of the dates are stored in the format we saw previously (dd mmm, yyyy), at least a couple are simply stored as the month and year (e.g. 'May 2019'). This means that the dates aren't all stored uniformly so we will have to take care when parsing them.

In [95]:
display(raw_steam_data['release_date'].value_counts().head())

steam_data['release_date'].value_counts().tail()

{'coming_soon': False, 'date': '13 Jul, 2018'}    65
{'coming_soon': False, 'date': ''}                64
{'coming_soon': False, 'date': '31 Jan, 2019'}    59
{'coming_soon': False, 'date': '5 Apr, 2016'}     59
{'coming_soon': False, 'date': '17 May, 2018'}    56
Name: release_date, dtype: int64

{'coming_soon': False, 'date': '10 Aug, 2011'}    1
{'coming_soon': False, 'date': '13 Aug, 2017'}    1
{'coming_soon': False, 'date': '24 Sep, 2007'}    1
{'coming_soon': False, 'date': '1 May, 2016'}     1
{'coming_soon': False, 'date': 'Oct 2015'}        1
Name: release_date, dtype: int64

Before we move on, let's quickly inspect some of the rows which have a blank date. 

It looks like some are special re-releases, like anniversary or game of the year editions, some are early access and not officially released yet, and others simply have a missing date. Apart from that there don't appear to be any clear patterns emerging, so as there are only 22 rows it may be best to remove them.

In [96]:
no_release_date = steam_data[steam_data['release_date'] == "{'coming_soon': False, 'date': ''}"]

print('Rows with no release date:', no_release_date.shape[0], '\n')
print_steam_links(no_release_date.head())
no_release_date.head()

Rows with no release date: 22 

Borderlands Game of the Year: https://store.steampowered.com/app/8980
Sherlock Holmes: The Mystery of the Persian Carpet: https://store.steampowered.com/app/11180
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby): https://store.steampowered.com/app/15540
The Great Art Race: https://store.steampowered.com/app/33580
SpellForce 2 - Anniversary Edition: https://store.steampowered.com/app/39550


,name,steam_appid,required_age,release_date,windows,mac,linux,price,english,developer,publisher
266,Borderlands Game of the Year,8980,18,"{'coming_soon': False, 'date': ''}",1,0,0,24.99,1,Gearbox Software,2K
319,Sherlock Holmes: The Mystery of the Persian Ca...,11180,3,"{'coming_soon': False, 'date': ''}",1,0,0,6.99,1,Frogwares,Frogwares
426,1... 2... 3... KICK IT! (Drop That Beat Like a...,15540,3,"{'coming_soon': False, 'date': ''}",1,0,0,6.99,1,"Dejobaan Games, LLC","Dejobaan Games, LLC"
731,The Great Art Race,33580,3,"{'coming_soon': False, 'date': ''}",1,0,0,3.99,1,Ascaron Entertainment ltd.,Assemble Entertainment
868,SpellForce 2 - Anniversary Edition,39550,3,"{'coming_soon': False, 'date': ''}",1,0,0,13.99,1,"Phenomic, THQ Nordic",THQ Nordic


Taking a look at the format of the column, we'll need to be using literal_eval once more. Apart from that it should be straightforward enough to extract the date.

In [97]:
print(type(steam_data['release_date'].iloc[0]))

steam_data['release_date'].iloc[0]

<class 'str'>


"{'coming_soon': False, 'date': '1 Nov, 2000'}"

In [98]:
print(type(literal_eval(steam_data['release_date'].iloc[0])))

literal_eval(steam_data['release_date'].iloc[0])['date']

<class 'dict'>


'1 Nov, 2000'

Once extracted, we can use the pd.to_datetime functon to interpret and store dates as datetime objects. This will be particularly useful as it will allow us to search and sort our dataset when it comes to performing analysis. Say for example we only wish to examine games released in 2010, by converting our dates to a python-recognisable format this will be very easy to achieve.

As seen below, we can supply the to_datetime function with our date and pandas will automatically interpret the format. We can then inspect it or print an attribute like the year. We can also provide pandas with the format explicitly, so it knows what to look for and how to parse it, which may be [quicker for large sets of data](https://stackoverflow.com/questions/32034689/why-is-pandas-to-datetime-slow-for-non-standard-time-format-such-as-2014-12-31).

In [99]:
timestamp = pd.to_datetime(literal_eval(steam_data['release_date'].iloc[0])['date'])

print(timestamp)
print(timestamp.year)

pd.to_datetime(literal_eval(steam_data['release_date'].iloc[0])['date'], format='%d %b, %Y')

2000-11-01 00:00:00
2000


Timestamp('2000-11-01 00:00:00')

Now we are ready to begin defining our function. As we only want to keep unreleased games, we first values from the coming_soon key, and keep only those where the value is False. Next we extract the release date, and set missing dates to np.nan, the default way of storing null values in pandas.

Then, using the formats we learned previously, we interpret those datesusing the to_datetime function. Once complete we pass over the dataframe once more with a general call to to_datetime, catching any dates we missed.

Finally we drop the columns we no longer need and return the dataframe.

Whilst functional, the process is quite slow. We can use the %timeit magic to test how long it takes to run our function, and we can see that on average it takes almost four seconds. Whilst manageable, we could certainly benefit from optimising our code, as this could quickly add up in larger data sets, where increasing efficiency can prove invaluable.

In [100]:
def process_release_date(df):
    df = df.copy()
    
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    # Only want to keep released games
    df = df[df['coming_soon'] == False].copy()
    
    # extract release date and set missing dates to null
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    df.loc[df['date'] == '', 'date'] = np.nan
    
    # Parse the date formats we have discovered
    df['datetime'] = pd.to_datetime(df['date'], format='%d %b, %Y', errors='ignore')
    df['datetime'] = pd.to_datetime(df['datetime'], format='%b %Y', errors='ignore')
    
    # Parse the rest of the date formats
    df['release_date'] = pd.to_datetime(df['datetime'])
    
    df = df.drop(['coming_soon', 'date', 'datetime'], axis=1)
    return df

%timeit process_release_date(steam_data)

3.72 s ± 45.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


There are a few areas we can investigate to make improvements. When initially parsing the date, we end up calling literal_eval twice, which may be a source of slowdown. We also loop over the entire dataset multiple times when calling the to_datetime function. 

We'll investigate which part is causing the greatest slowdown, but we can be certain that reducing the traversals over the data set will most likely provide significant gains. There are also a few other issues that we'll dive into over the course of our optimisation process.

First, let's find out where the main slowdowns are. As we just saw we can use the %timeit magic to time our function. We can also use the in-built time module to inspect parts of our code.

In [101]:
def process_release_date(df):
    df = df.copy()
    
    eval_start = time.time()
    
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    df = df[df['coming_soon'] == False].copy()
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    
    print('Evaluation run-time:', time.time() - eval_start)
    
    df.loc[df['date'] == '', 'date'] = None
    
    first_parse_start = time.time()
    
    df['datetime'] = pd.to_datetime(df['date'], format='%d %b, %Y', errors='ignore')
    df['datetime'] = pd.to_datetime(df['datetime'], format='%b %Y', errors='ignore')
    
    print('First parse run-time:', time.time() - first_parse_start)
    
    second_parse_start = time.time()
    
    df['release_date'] = pd.to_datetime(df['datetime'])
    
    print('Final parse run-time:', time.time() - second_parse_start)
    
    df = df.drop(['coming_soon', 'date', 'datetime'], axis=1)
    return df

function_start = time.time()
process_release_date(steam_data)
print('\nTotal run-time:', time.time() - function_start)

Evaluation run-time: 0.7607526779174805
First parse run-time: 0.008997678756713867
Final parse run-time: 2.897845506668091

Total run-time: 3.7045834064483643


Immediately we can see that the majority of run-time is taken up by the final call to pd.to_datetime. This suggests that the first two calls are not functioning as expected - they are possibly terminating after the first error instead of skipping over it as desired - and most of the work is being done by the final call. Now it makes sense why it is slow - pandas has to figure out how each date is formatted, and since we know we have some variations this may be slowing it down considerably.

Whilst the evaluation run-time is much shorter, our multiple calls to literal_eval may be slowing the function as well, so we may wish to investigate that. As we know the biggest slowdown, we should begin there.

We now know that handling our dates in their current form is slow, and we know that we have some different formats mixed in there. Whilst there are likely many possible solutions to this problem, using regular expressions (or regex) comes to mind as they tend to excel at pattern matching in strings.

We know for sure two of the patterns, so let's build a regex for each of those. Then we can iteratively add more as we discover any other patterns. A powerful and useful tool for building and testing regex can be found at [regexr.com](https://regexr.com/).

In [102]:
pattern = r'[\d]{1,2} [A-Za-z]{3}, [\d]{4}'
string = '13 Jul, 2018'

print(re.search(pattern, string))

pattern = r'[A-Za-z]{3} [\d]{4}'
string = 'Apr 2016'

print(re.search(pattern, string))

<re.Match object; span=(0, 12), match='13 Jul, 2018'>
<re.Match object; span=(0, 8), match='Apr 2016'>


Using these two patterns we can start building out our function. We're going to apply a function to the date column which searches for each pattern, returning a standardised date string which we will then feed into the to_datetime function.

Our first search matches the 'mmm yyyy' pattern, like 'Apr 2019'. As we don't know the particular day for these matches we will assume it is the first of the month, returning '1 Apr 2019' in this example.

If we don't match this, we'll check for the second case. Our second match will be the 'dd mmm, yyyy' pattern, like '13 Jul, 2018'. In this case we will simply return the match with the comma removed, to become '13 Jul 2018'.

Finally we'll check for the empty string, and return it for now.

For anything else we'll simply print the string so we know what else we should be searching for.

In [103]:
def process_release_date(df):
    df = df.copy()
    
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    df = df[df['coming_soon'] == False].copy()
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    
    def parse_date(x):
        if re.search(r'[A-Za-z]{3} [\d]{4}', x):
            return '1 ' + x 
        elif re.search(r'[\d]{1,2} [A-Za-z]{3}, [\d]{4}', x):
            return x.replace(',', '')
        elif x == '':
            return x
        else:
            print(x)
            
    df['date'] = df['date'].apply(parse_date)
    df['release_date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    
    df = df.drop(['coming_soon', 'date'], axis=1)
    
    return df

result = process_release_date(steam_data)

It looks like we've caught all of the patterns and don't have any to take care of.

Previously we used the `infer_datetime_format` parameter of to_datetime, which can speed up the process. However, as we now know exactly the format our dates will be in, we can explicitly set it ourselves, which should be the fastest way of doing things.

We also need to decide how to handle our missing dates - those with the empty strings. For now let's change the way the function handles errors from raise to coerce, which returns NaT (not a time) instead.

We can now rewrite our function and time it as we did before.

In [104]:
def process_release_date_old(df):
    df = df.copy()
    
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    df = df[df['coming_soon'] == False].copy()
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    
    # Simple parsing
    df['release_date'] = pd.to_datetime(df['date'])
    
    df = df.drop(['coming_soon', 'date'], axis=1)
    
    return df


def process_release_date_new(df):
    df = df.copy()
    
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    df = df[df['coming_soon'] == False].copy()
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    
    # Complex parsing
    def parse_date(x):
        if re.search(r'[A-Za-z]{3} [\d]{4}', x):
            return '1 ' + x
        elif re.search(r'[\d]{1,2} [A-Za-z]{3}, [\d]{4}', x):
            return x.replace(',', '')
        elif x == '':
            return x
        else:
            # Should be everything, print out anything left just in case
            print(x)
            
    df['date'] = df['date'].apply(parse_date)
    df['release_date'] = pd.to_datetime(df['date'], format='%d %b %Y', errors='coerce')
    
    df = df.drop(['coming_soon', 'date'], axis=1)
    
    return df

print('Testing date parsing:\n')
%timeit process_release_date_old(steam_data)
%timeit process_release_date_new(steam_data)

Testing date parsing:

3.69 s ± 72.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
917 ms ± 7.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Our results show that the new method is almost four times faster, so we're on the right track.

Another optimisation we can make here is checking which part of the if/elif statements has the most matches. It makes sense to order our statements from most matches to least, so for the majority of rows we only have to search through once. 

To do this, instead of returning the date we'll return a number for each match. We can then print the value counts for the column and see which is the most frequent.

In [105]:
def optimise_regex_order(df):
    df = df.copy()
    
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    df = df[df['coming_soon'] == False].copy()
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    
    def parse_date(x):
        if re.search(r'[A-Za-z]{3} [\d]{4}', x):
            return '0: mmm yyyy' # '1 ' + x
        elif re.search(r'[\d]{1,2} [A-Za-z]{3}, [\d]{4}', x):
            return '1: dd mmm, yyyy' # x.replace(',', '')
        elif x == '':
            return '2: empty' # pass
        else:
            # Should be everything, print out anything left just in case
            print(x)
            
    df['release_date'] = df['date'].apply(parse_date)
    
    return df


result = optimise_regex_order(steam_data)

result['release_date'].value_counts()

1: dd mmm, yyyy    27294
0: mmm yyyy           57
2: empty              22
Name: release_date, dtype: int64

By far the majority of dates are in the 'dd mmm, yyyy' format, which is second in our if/else statements. This means that for all these rows we are unnecessarily searching the string twice. Simply by reordering our searches we should see a minor performance improvement.

In [106]:
def process_release_date_unordered(df):
    df = df.copy()
    
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    df = df[df['coming_soon'] == False].copy()
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    
    def parse_date(x):
        if re.search(r'[A-Za-z]{3} [\d]{4}', x):
            return '1 ' + x
        elif re.search(r'[\d]{1,2} [A-Za-z]{3}, [\d]{4}', x):
            return x.replace(',', '')
        elif x == '':
            return x
        else:
            # Should be everything, print out anything left just in case
            print(x)
            
    df['release_date'] = df['date'].apply(parse_date)
    df['release_date'] = pd.to_datetime(df['date'], format='%d %b %Y', errors='coerce')
    df = df.drop(['coming_soon', 'date'], axis=1)
    
    return df


def process_release_date_ordered(df):
    df = df.copy()
    
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    df = df[df['coming_soon'] == False].copy()
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    
    def parse_date(x):
        if re.search(r'[\d]{1,2} [A-Za-z]{3}, [\d]{4}', x):
            return x.replace(',', '')
        elif re.search(r'[A-Za-z]{3} [\d]{4}', x):
            return '1 ' + x
        elif x == '':
            return x
        else:
            # Should be everything, print out anything left just in case
            print(x)
            
    df['release_date'] = df['date'].apply(parse_date)
    df['release_date'] = pd.to_datetime(df['date'], format='%d %b %Y', errors='coerce')
    df = df.drop(['coming_soon', 'date'], axis=1)
    
    return df


%timeit process_release_date_unordered(steam_data)
%timeit process_release_date_ordered(steam_data)

834 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
809 ms ± 9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It's an improvement, if only slightly, so we'll keep it. If anything this goes to show how fast regex pattern matching is, as there was hardly any slowdown in searching every string twice.

Now parsing is well-optimised we can move on to the evaluation section.

In [107]:
# Testing evaluation methods
def evaluation_method_original(df):
    df = df.copy()
    
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])    
    df = df[df['coming_soon'] == False].copy()
    df['release_date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    
    return df


def evaluation_method_1(df):
    df = df.copy()
    
    df['release_date'] = df['release_date'].apply(lambda x: literal_eval(x))
    
    df['coming_soon'] = df['release_date'].apply(lambda x: x['coming_soon'])
    df = df[df['coming_soon'] == False].copy()
    
    df['release_date'] = df['release_date'].apply(lambda x: x['date'])
    
    return df


def evaluation_method_2(df):
    df = df.copy()
    
    df['release_date'] = df['release_date'].apply(lambda x: literal_eval(x))
    df_2 = df['release_date'].transform([lambda x: x['coming_soon'], lambda x: x['date']])
    df = pd.concat([df, df_2], axis=1)
    
    return df


def evaluation_method_3(df):
    df = df.copy()
    
    def eval_date(x):
        x = literal_eval(x)
        if x['coming_soon']:
            return np.nan
        else:
            return x['date']
    
    df['release_date'] = df['release_date'].apply(eval_date)
    df = df[df['release_date'].notnull()]  # could change to drop when '' and deal with missing release dates also
    
    return df


%timeit evaluation_method_original(steam_data)

%timeit evaluation_method_1(steam_data)
%timeit evaluation_method_2(steam_data)
%timeit evaluation_method_3(steam_data)

734 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
394 ms ± 2.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
387 ms ± 4.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
373 ms ± 2.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It looks like we may have been right in our assumption that multiple calls to literal_eval were slowing down the function - by calling it once instead of twice we almost halved the run-time.

Of our new methods the final one was just about the fastest, which is useful because it contains flexible custom logic we can modify if needed. Let's put everything together into our final function, and time it once more to see the improvements we've made.

We'll make a couple of changes so we can easily remove missing values at the end, which should mean we end up with clean release dates.

In [108]:
def process_release_date(df):
    df = df.copy()
    
    def eval_date(x):
        x = literal_eval(x)
        if x['coming_soon']:
            return '' # return blank string so can drop missing at end
        else:
            return x['date']
    
    df['release_date'] = df['release_date'].apply(eval_date)
    
    def parse_date(x):
        if re.search(r'[\d]{1,2} [A-Za-z]{3}, [\d]{4}', x):
            return x.replace(',', '')
        elif re.search(r'[A-Za-z]{3} [\d]{4}', x):
            return '1 ' + x
        elif x == '':
            return np.nan
        else:
            # Should be everything, print out anything left just in case
            print(x)
            
    df['release_date'] = df['release_date'].apply(parse_date)
    df['release_date'] = pd.to_datetime(df['release_date'], format='%d %b %Y', errors='coerce')
    
    df = df[df['release_date'].notnull()]
    
    return df

%timeit process_release_date(steam_data)

511 ms ± 4.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Referring back to our original time of 3.6s, we've achieved a 7x speed increase. That's almost an order of magnitude improvement. 

We'll now update our process function, run it on our data set, and move on to some final checks.

In [109]:
def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""
    
    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()
    
    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()
    
    # Remove collumns with more than 50% null values
    df = process_null_cols(df)
    
    # Process rest of columns
    df = df.drop(['achievements', 'content_descriptors'], axis=1)
    df = process_type(df)
    df = process_name(df)
    df = process_age(df)
    df = process_platforms(df)
    df = process_price(df)
    df = process_language(df)
    df = process_developers_and_publishers(df)
    df = process_release_date(df)
    
    # Process columns which export data
    df = process_descriptions(df, export=True)
    df = process_images(df, export=True)
    df = process_info(df, export=True)
    df = process_requirements(df, export=True)
    df = process_categories(df, export=True)
    df = process_genres(df, export=True)
    
    return df

steam_data = process(raw_steam_data)
steam_data.head()

Exported description data to '../data/exports/steam_description_data.csv'
Exported image data to '../data/exports/steam_image_data.csv'
Exported support info to '../data/exports/steam_support_info.csv'
Exported requirements data to '../data/exports/steam_requirements_data.csv'
Exported category data to '../data/exports/steam_category_data.csv'
Exported genre data to '../data/exports/steam_genre_data.csv'


,name,steam_appid,required_age,release_date,windows,mac,linux,price,english,developer,publisher
0,Counter-Strike,10,3,2000-11-01,1,1,1,7.19,1,Valve,Valve
1,Team Fortress Classic,20,3,1999-04-01,1,1,1,3.99,1,Valve,Valve
2,Day of Defeat,30,3,2003-05-01,1,1,1,3.99,1,Valve,Valve
3,Deathmatch Classic,40,3,2001-06-01,1,1,1,3.99,1,Valve,Valve
4,Half-Life: Opposing Force,50,3,1999-11-01,1,1,1,3.99,1,Gearbox Software,Valve


### Final Steps

Our data set is hopefully complete. Before we export it to csv, let's check if we have any null values.

In [110]:
steam_data.isnull().sum()

name            0
steam_appid     0
required_age    0
release_date    0
windows         0
mac             0
linux           0
price           0
english         0
developer       0
publisher       0
dtype: int64

We will also export a data set including the category and genre data, so let's also check those for missing values.

We'll also prepend the columns in each data set to make them easily identifiable.

In [111]:
genre_data = pd.read_csv('../data/exports/steam_genre_data.csv')

# prepend with 'genre_' to make easily identifiable
genre_data = genre_data.add_prefix('genre_').rename({'genre_steam_appid':'steam_appid'}, axis=1)
genre_data.isnull().sum()

steam_appid                      0
genre_accounting                 0
genre_action                     0
genre_adventure                  0
genre_animation_and_modeling     0
genre_audio_production           0
genre_casual                     0
genre_design_and_illustration    0
genre_documentary                0
genre_early_access               0
genre_education                  0
genre_free_to_play               0
genre_game_development           0
genre_gore                       0
genre_indie                      0
genre_massively_multiplayer      0
genre_nudity                     0
genre_photo_editing              0
genre_rpg                        0
genre_racing                     0
genre_sexual_content             0
genre_simulation                 0
genre_software_training          0
genre_sports                     0
genre_strategy                   0
genre_tutorial                   0
genre_utilities                  0
genre_video_production           0
genre_violent       

In [112]:
category_data = pd.read_csv('../data/exports/steam_category_data.csv')
category_data = category_data.add_prefix('category_').rename({'category_steam_appid':'steam_appid'}, axis=1)
category_data.isnull().sum()

steam_appid                            0
category_captions_available            0
category_co_op                         0
category_commentary_available          0
category_cross_platform_multiplayer    0
category_full_controller_support       0
category_in_app_purchases              0
category_includes_source_sdk           0
category_includes_level_editor         0
category_local_co_op                   0
category_local_multi_player            0
category_mmo                           0
category_mods                          0
category_mods_require_hl2              0
category_multi_player                  0
category_online_co_op                  0
category_online_multi_player           0
category_partial_controller_support    0
category_shared_or_split_screen        0
category_single_player                 0
category_stats                         0
category_steam_achievements            0
category_steam_cloud                   0
category_steam_leaderboards            0
category_steam_t

Looks good. We also want to check that no games slipped through that aren't released yet (data scraped on or before 1st May 2019).

In [113]:
steam_data[steam_data['release_date'] > '2019-05-01']

,name,steam_appid,required_age,release_date,windows,mac,linux,price,english,developer,publisher


### Combining and exporting data frames

Now that we're happy with our dataframe we are ready to export to file and finish this part of the project. 

First we export steam_data, then we merge genre_data and category_data into a new dataframe, check it for missing values, then export it.

In [114]:
steam_data.to_csv('../data/steam_data_clean.csv', index=False)

steam_data_full = steam_data.merge(genre_data, how='left', on='steam_appid')
steam_data_full = steam_data_full.merge(category_data, how='left', on='steam_appid')

null_counts = steam_data_full.isnull().sum()
print(null_counts[null_counts > 0].shape[0])

steam_data_full.to_csv('../data/steam_data_with_genre_and_category.csv', index=False)

steam_data_full.head()

0


,name,steam_appid,required_age,release_date,windows,mac,linux,price,english,developer,publisher,genre_accounting,genre_action,genre_adventure,genre_animation_and_modeling,genre_audio_production,genre_casual,genre_design_and_illustration,genre_documentary,genre_early_access,genre_education,genre_free_to_play,genre_game_development,genre_gore,genre_indie,genre_massively_multiplayer,genre_nudity,genre_photo_editing,genre_rpg,genre_racing,genre_sexual_content,genre_simulation,genre_software_training,genre_sports,genre_strategy,genre_tutorial,genre_utilities,genre_video_production,genre_violent,genre_web_publishing,category_captions_available,category_co_op,category_commentary_available,category_cross_platform_multiplayer,category_full_controller_support,category_in_app_purchases,category_includes_source_sdk,category_includes_level_editor,category_local_co_op,category_local_multi_player,category_mmo,category_mods,category_mods_require_hl2,category_multi_player,category_online_co_op,category_online_multi_player,category_partial_controller_support,category_shared_or_split_screen,category_single_player,category_stats,category_steam_achievements,category_steam_cloud,category_steam_leaderboards,category_steam_trading_cards,category_steam_turn_notifications,category_steam_workshop,category_steamvr_collectibles,category_vr_support,category_valve_anti_cheat_enabled
0,Counter-Strike,10,3,2000-11-01,1,1,1,7.19,1,Valve,Valve,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Team Fortress Classic,20,3,1999-04-01,1,1,1,3.99,1,Valve,Valve,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Day of Defeat,30,3,2003-05-01,1,1,1,3.99,1,Valve,Valve,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Deathmatch Classic,40,3,2001-06-01,1,1,1,3.99,1,Valve,Valve,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,Half-Life: Opposing Force,50,3,1999-11-01,1,1,1,3.99,1,Gearbox Software,Valve,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


# Next steps

We could clean some of the data we exported, like description and requirements.

We are now ready to move on to cleaning our steamspy data.